In [1]:
# Set columns in display
# pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_columns') # Reset options to default

# Set rows in display
# pd.set_option('display.max_rows', None)
# pd.reset_option('display.max_rows') # Reset options to default

# Load Util File with TARE Model Functions

In [2]:
import os

# Measure Package 0: Baseline
menu_mp = 0
input_mp = 'baseline'

# Get the current working directory of the project
project_root = os.path.abspath(os.getcwd())
print(f"Project root directory: {project_root}")

# Relative path to the file from the project root
relative_path = r"tare_model_functions_v1_5_1.ipynb"

# Construct the absolute path to the file
file_path = os.path.join(project_root, relative_path)
print(f"File path: {file_path}")

# Run the notebook and import variables
if os.path.exists(relative_path):
    get_ipython().run_line_magic('run', f'-i "{relative_path}"')
    print("Loaded All TARE Model Functions")
else:
    print(f"File not found: {relative_path}")

Project root directory: c:\Users\14128\Research\cmu-tare-model
File path: c:\Users\14128\Research\cmu-tare-model\tare_model_functions_v1_5_1.ipynb
Loaded All TARE Model Functions


In [3]:
# Storing Result Outputs in output_results folder
relative_path = r"output_results"
output_folder_path = os.path.join(project_root, relative_path)
print(f"Result outputs will be exported here: {output_folder_path}")

Result outputs will be exported here: c:\Users\14128\Research\cmu-tare-model\output_results


# Simulate Residential Energy Consumption using NREL End-Use Savings Shapes
- Filter EUSS Data: Only occupied units and Single Family Homes



In [4]:
# The ``inline`` flag will use the appropriate backend to make figures appear inline in the notebook.  
%matplotlib inline

import pandas as pd
import numpy as np

# `plt` is an alias for the `matplotlib.pyplot` module
import matplotlib.pyplot as plt

# import seaborn library (wrapper of matplotlib)
import seaborn as sns
sns.set(style="darkgrid")

# For regex, import re
import re

from datetime import datetime

# Get the current datetime
# Start the timer
start_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [5]:
print("""
-------------------------------------------------------------------------------------------------------
Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
-------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------
BASELINE (Measure Package 0)
-------------------------------------------------------------------------------------------------------
""")

# Measure Package 0: Baseline
menu_mp = 0
input_mp = 'baseline'

filename = "baseline_metadata_and_annual_results.csv"
relative_path = os.path.join(r"euss_data\resstock_amy2018_release_1.1\state", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

print("""
-------------------------------------------------------------------------------------------------------
Data Filters: Only occupied units and Single Family Homes
-------------------------------------------------------------------------------------------------------
""")

# Fix DtypeWarning error in columns:
# 'in.neighbors', 'in.geometry_stories_low_rise', 'in.iso_rto_region', 'in.pv_orientation', 'in.pv_system_size'
columns_to_string = {11: str, 61: str, 121: str, 103: str, 128: str, 129: str}
df_euss_am_baseline = pd.read_csv(file_path, dtype=columns_to_string, index_col="bldg_id") # UPDATE: Set index to 'bldg_id'

# Filter for occupied homes
occupancy_filter = df_euss_am_baseline['in.vacancy_status'] == 'Occupied'
df_euss_am_baseline = df_euss_am_baseline.loc[occupancy_filter]

# Filter for single family home building type
house_type_list = ['Single-Family Attached', 'Single-Family Detached']
house_type_filter = df_euss_am_baseline['in.geometry_building_type_recs'].isin(house_type_list)
df_euss_am_baseline = df_euss_am_baseline.loc[house_type_filter]
# df_euss_am_baseline


-------------------------------------------------------------------------------------------------------
Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
-------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------
BASELINE (Measure Package 0)
-------------------------------------------------------------------------------------------------------

Retrieved data for filename: baseline_metadata_and_annual_results.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\euss_data\resstock_amy2018_release_1.1\state\baseline_metadata_and_annual_results.csv

------------------------

In [6]:
# # Make a copy of the dataframe
# df_euss_am_baseline = df_euss_am_baseline.copy()

# Choose between national or sub-national level analysis
menu_state = get_menu_choice(menu_prompt, {'N', 'Y'})   # This code is only run in baseline

# National Level 
if menu_state == 'N':
    print("You chose to analyze all of the United States.")
    input_state = 'National'
    location_id = 'National'

# Filter down to state or city
else:
    input_state = get_state_choice(df_euss_am_baseline)    
    print(f"You chose to filter for: {input_state}")

    location_id = str(input_state)
    print(f"Location ID is: {location_id}")


    state_filter = df_euss_am_baseline['in.state'].eq(input_state)
    df_euss_am_baseline = df_euss_am_baseline.loc[state_filter]

    print(city_prompt)
    print(df_euss_am_baseline['in.city'].value_counts())

    menu_city = get_menu_choice(city_menu_prompt, {'N', 'Y'})

    # Filter for the entire selected state
    if menu_city == 'N':
        print(f"You chose to analyze all of state: {input_state}")
        
        location_id = str(input_state)
        print(f"Location ID is: {location_id}")
        
    # Filter to a city within the selected state
    else:
        input_cityFilter = get_city_choice(df_euss_am_baseline, input_state)
        print(f"You chose to filter for: {input_state}, {input_cityFilter}")

        location_id = input_cityFilter.replace(', ', '_').strip()
        print(f"Location ID is: {location_id}")

        city_filter = df_euss_am_baseline['in.city'].eq(f"{input_state}, {input_cityFilter}")
        df_euss_am_baseline = df_euss_am_baseline.loc[city_filter]

# Display the filtered dataframe
df_euss_am_baseline

You chose to analyze all of the United States.


,upgrade,weight,applicability,in.sqft,in.ahs_region,in.ashrae_iecc_climate_zone_2004,in.ashrae_iecc_climate_zone_2004_2_a_split,in.bathroom_spot_vent_hour,in.bedrooms,in.building_america_climate_zone,...,out.emissions.natural_gas.lrmer_low_re_cost_25_2025_start.co2e_kg,out.emissions.propane.lrmer_low_re_cost_25_2025_start.co2e_kg,out.emissions.electricity.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.fuel_oil.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.natural_gas.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.propane.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_95_decarb_by_2035_15_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_low_re_cost_15_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_low_re_cost_25_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_mid_case_15_2025_start.co2e_kg
bldg_id,,,,,,,,,,,,,,,,,,,,,
239,0,242.131013,True,1690.0,Non-CBSA East South Central,3A,3A,Hour20,3,Hot-Humid,...,215.943534,0.000000,8773.384074,0.0,215.943534,0.000000,3565.038262,6416.193347,5755.373221,8989.327608
273,0,242.131013,True,1690.0,Non-CBSA East South Central,3A,3A,Hour12,3,Mixed-Humid,...,0.000000,0.000000,11296.731129,0.0,0.000000,0.000000,4339.903757,8193.678510,7254.665194,11296.731129
307,0,242.131013,True,1220.0,Non-CBSA East South Central,3A,3A,Hour0,4,Hot-Humid,...,0.000000,0.000000,8750.011820,0.0,0.000000,0.000000,3345.187937,6249.625611,5587.946834,8750.011820
409,0,242.131013,True,1220.0,Non-CBSA East South Central,3A,3A,Hour20,2,Hot-Humid,...,1642.477930,0.000000,5725.103641,0.0,1642.477930,0.000000,3784.993820,5489.549041,5164.458936,7367.581571
517,0,242.131013,True,1220.0,Non-CBSA East South Central,3A,3A,Hour1,3,Mixed-Humid,...,0.000000,0.000000,8932.439414,0.0,0.000000,0.000000,3441.414837,6415.303853,5721.073473,8932.439414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548226,0,242.131013,True,2176.0,Non-CBSA Mountain,6B,6B,Hour3,4,Cold,...,15324.356044,0.000000,2313.298407,0.0,15324.356044,0.000000,16802.023908,16545.095582,16472.920871,17637.654451
548228,0,242.131013,True,1690.0,Non-CBSA Mountain,6B,6B,Hour6,4,Cold,...,8192.601682,0.000000,1889.439924,0.0,8192.601682,0.000000,9394.122057,9195.903552,9129.578822,10082.041606
548417,0,242.131013,True,885.0,Non-CBSA Mountain,6B,6B,Hour18,2,Cold,...,5212.758359,0.000000,2112.907195,0.0,5212.758359,0.000000,6546.826589,6318.763521,6253.986448,7325.665554


## Project Future Energy Consumption Using EIA Heating Degree Day (HDD) Forecasted Data (Factors)

In [7]:
# Factors for 2022 to 2050
filename = 'aeo_projections_2022_2050.xlsx'
relative_path = os.path.join(r"projections", filename)
file_path = os.path.join(project_root, relative_path)
df_hdd_projection_factors = pd.read_excel(io=file_path, sheet_name='hdd_factors_2022_2050')

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# Convert the factors dataframe into a lookup dictionary
hdd_factor_lookup = df_hdd_projection_factors.set_index(['census_division']).to_dict('index')
hdd_factor_lookup

Retrieved data for filename: aeo_projections_2022_2050.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\projections\aeo_projections_2022_2050.xlsx


{'National': {2022: 1,
  2023: 1.0028349414260749,
  2024: 0.9389536266963965,
  2025: 0.9344844368179533,
  2026: 0.9300828169743566,
  2027: 0.9257070259326153,
  2028: 0.9212779053519207,
  2029: 0.9168538581973203,
  2030: 0.9124351151864318,
  2031: 0.9080377039911245,
  2032: 0.9036541297129915,
  2033: 0.8992492812396443,
  2034: 0.894875507855348,
  2035: 0.8904758482849783,
  2036: 0.8860390020882589,
  2037: 0.8817285900905196,
  2038: 0.877365886428882,
  2039: 0.8729314040841085,
  2040: 0.8685839209369028,
  2041: 0.8642702226890459,
  2042: 0.8599120736340495,
  2043: 0.8555441810694344,
  2044: 0.8511753084862802,
  2045: 0.8468232704962843,
  2046: 0.8425090534289743,
  2047: 0.8382247585710751,
  2048: 0.8339389072548168,
  2049: 0.8297055204635582,
  2050: 0.8255002687057338},
 'East North Central': {2022: 1,
  2023: 0.9811731756651626,
  2024: 0.9307608526528707,
  2025: 0.928426948809709,
  2026: 0.9262486385560915,
  2027: 0.9239147347129298,
  2028: 0.921580830869

In [8]:
print("""
-------------------------------------------------------------------------------------------------------
PROJECT FUTURE ENERGY CONSUMPTION: Baseline Consumption
-------------------------------------------------------------------------------------------------------
Creating dataframe to store annual energy consumption calculations ...
""")

# df_baseline_enduse(df_baseline, df_enduse, category, fuel_filter='Yes', tech_filter='Yes')
df_euss_am_baseline_home = df_enduse_refactored(df_baseline = df_euss_am_baseline,
                                                fuel_filter = 'Yes',
                                                tech_filter = 'Yes')

# Make a copy of the dataframe
df_baseline_scenario_consumption = df_euss_am_baseline_home.copy()

# Project Future Energy Consumption
df_euss_am_baseline_home, df_baseline_scenario_consumption = project_future_consumption(df=df_euss_am_baseline_home,
                                                                                        hdd_factor_lookup=hdd_factor_lookup,
                                                                                        menu_mp=menu_mp
                                                                                        )
# Display the baseline scenario summary dataframe
df_euss_am_baseline_home


-------------------------------------------------------------------------------------------------------
PROJECT FUTURE ENERGY CONSUMPTION: Baseline Consumption
-------------------------------------------------------------------------------------------------------
Creating dataframe to store annual energy consumption calculations ...

Processing column: in.clothes_dryer
Initial data types: object
Data types after processing: object
Processing column: in.cooking_range
Initial data types: object
Data types after processing: object
331531 rows remain after applying total heating consumption calculation
Filtered for the following fuels: ['Natural Gas', 'Electricity', 'Propane', 'Fuel Oil']
321357 rows remain after applying heating fuel filter
Filtered for the following Heating technologies: ['Electricity ASHP', 'Electricity Baseboard', 'Electricity Electric Boiler', 'Electricity Electric Furnace', 'Fuel Oil Fuel Boiler', 'Fuel Oil Fuel Furnace', 'Natural Gas Fuel Boiler', 'Natural Gas Fuel

,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,gea_region,state,city,county,...,baseline_2029_cooking_consumption,baseline_2030_cooking_consumption,baseline_2031_cooking_consumption,baseline_2032_cooking_consumption,baseline_2033_cooking_consumption,baseline_2034_cooking_consumption,baseline_2035_cooking_consumption,baseline_2036_cooking_consumption,baseline_2037_cooking_consumption,baseline_2038_cooking_consumption
bldg_id,,,,,,,,,,,,,,,,,,,,,
239,1690.0,South,East South Central,East South Central,Hot-Humid,90,SRSOc,AL,Not in a census Place,G0100390,...,979.44,979.44,979.44,979.44,979.44,979.44,979.44,979.44,979.44,979.44
273,1690.0,South,East South Central,East South Central,Mixed-Humid,90,SRSOc,AL,In another census Place,G0100150,...,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20
307,1220.0,South,East South Central,East South Central,Hot-Humid,90,SRSOc,AL,Not in a census Place,G0100850,...,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20
409,1220.0,South,East South Central,East South Central,Hot-Humid,90,SRSOc,AL,Not in a census Place,G0100050,...,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63
517,1220.0,South,East South Central,East South Central,Mixed-Humid,89,SRSOc,AL,In another census Place,G0101270,...,377.18,377.18,377.18,377.18,377.18,377.18,377.18,377.18,377.18,377.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548109,1690.0,West,Mountain,Mountain North,Cold,23,RMPAc,WY,In another census Place,G5600050,...,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61
548226,2176.0,West,Mountain,Mountain North,Cold,23,RMPAc,WY,In another census Place,G5600050,...,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61
548228,1690.0,West,Mountain,Mountain North,Cold,24,RMPAc,WY,Not in a census Place,G5600010,...,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61


In [9]:
# Display the projected future energy consumption dataframe
df_baseline_scenario_consumption

,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,gea_region,state,city,county,...,baseline_2029_cooking_consumption,baseline_2030_cooking_consumption,baseline_2031_cooking_consumption,baseline_2032_cooking_consumption,baseline_2033_cooking_consumption,baseline_2034_cooking_consumption,baseline_2035_cooking_consumption,baseline_2036_cooking_consumption,baseline_2037_cooking_consumption,baseline_2038_cooking_consumption
bldg_id,,,,,,,,,,,,,,,,,,,,,
239,1690.0,South,East South Central,East South Central,Hot-Humid,90,SRSOc,AL,Not in a census Place,G0100390,...,979.44,979.44,979.44,979.44,979.44,979.44,979.44,979.44,979.44,979.44
273,1690.0,South,East South Central,East South Central,Mixed-Humid,90,SRSOc,AL,In another census Place,G0100150,...,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20
307,1220.0,South,East South Central,East South Central,Hot-Humid,90,SRSOc,AL,Not in a census Place,G0100850,...,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20
409,1220.0,South,East South Central,East South Central,Hot-Humid,90,SRSOc,AL,Not in a census Place,G0100050,...,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63
517,1220.0,South,East South Central,East South Central,Mixed-Humid,89,SRSOc,AL,In another census Place,G0101270,...,377.18,377.18,377.18,377.18,377.18,377.18,377.18,377.18,377.18,377.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548109,1690.0,West,Mountain,Mountain North,Cold,23,RMPAc,WY,In another census Place,G5600050,...,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61
548226,2176.0,West,Mountain,Mountain North,Cold,23,RMPAc,WY,In another census Place,G5600050,...,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61
548228,1690.0,West,Mountain,Mountain North,Cold,24,RMPAc,WY,Not in a census Place,G5600010,...,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61


# Public Perspective: Monetized Marginal Damages from Emissions

## Fossil Fuels: Climate and Health-Related Pollutants

In [10]:
# Print header
print(""" 
-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: FOSSIL FUELS
-------------------------------------------------------------------------------------------------------
Fossil Fuels (Natural Gas, Fuel Oil, Propane):
- NOx, SO2, CO2: 
    - RESNET Table 7.1.2 Emissions Factors for Household Combustion Fuels
    - Source: https://www.resnet.us/wp-content/uploads/ANSIRESNETICC301-2022_resnetpblshd.pdf
    - All factors are in units of lb/Mbtu; energy consumption in kWh needs conversion.
- PM2.5: 
    - A National Methodology and Emission Inventory for Residential Fuel Combustion
    - Source: https://www3.epa.gov/ttnchie1/conference/ei12/area/haneke.pdf
-------------------------------------------------------------------------------------------------------
""")

# Calculate emission factors for each fuel type
fuel_oil_factors = calculate_fossil_fuel_emission_factor(
    fuel_type="fuelOil", so2_factor=0.0015, nox_factor=0.1300, pm25_factor=0.83,
    conversion_factor1=1000, conversion_factor2=138500
)
natural_gas_factors = calculate_fossil_fuel_emission_factor(
    fuel_type="naturalGas", so2_factor=0.0006, nox_factor=0.0922, pm25_factor=1.9,
    conversion_factor1=1_000_000, conversion_factor2=1039
)
propane_factors = calculate_fossil_fuel_emission_factor(
    fuel_type="propane", so2_factor=0.0002, nox_factor=0.1421, pm25_factor=0.17,
    conversion_factor1=1000, conversion_factor2=91452
)

# Combine all factors
all_factors = {**fuel_oil_factors, **natural_gas_factors, **propane_factors}

# Create DataFrame
df_marg_emis_factors = pd.DataFrame.from_dict(all_factors, orient="index", columns=["value"])
df_marg_emis_factors.reset_index(inplace=True)
df_marg_emis_factors.columns = ["pollutant", "value"]

# Split pollutant into fuel type and pollutant name
df_marg_emis_factors[["fuel_type", "pollutant"]] = df_marg_emis_factors["pollutant"].str.split("_", expand=True)

# Update the units to metric tons per kWh
df_marg_emis_factors["unit"] = "[mt/kWh]"

# Convert units from lb/kWh to metric tons/kWh where applicable
lb_to_mt = 0.00045359237
pollutants_in_lb = ['so2', 'nox', 'pm25']
df_marg_emis_factors['value'] = df_marg_emis_factors.apply(
    lambda row: row['value'] * lb_to_mt if row['pollutant'] in pollutants_in_lb else row['value'], axis=1
)

# Add 'state' column with default value
df_marg_emis_factors = df_marg_emis_factors.assign(state="National")

# Reorder columns for clarity
df_marg_emis_factors = df_marg_emis_factors[["state", "fuel_type", "pollutant", "value", "unit"]]
df_marg_emis_factors

 
-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: FOSSIL FUELS
-------------------------------------------------------------------------------------------------------
Fossil Fuels (Natural Gas, Fuel Oil, Propane):
- NOx, SO2, CO2: 
    - RESNET Table 7.1.2 Emissions Factors for Household Combustion Fuels
    - Source: https://www.resnet.us/wp-content/uploads/ANSIRESNETICC301-2022_resnetpblshd.pdf
    - All factors are in units of lb/Mbtu; energy consumption in kWh needs conversion.
- PM2.5: 
    - A National Methodology and Emission Inventory for Residential Fuel Combustion
    - Source: https://www3.epa.gov/ttnchie1/conference/ei12/area/haneke.pdf
-------------------------------------------------------------------------------------------------------



,state,fuel_type,pollutant,value,unit
0,National,fuelOil,so2,2.321486e-09,[mt/kWh]
1,National,fuelOil,nox,2.011954e-07,[mt/kWh]
2,National,fuelOil,pm25,9.274769e-09,[mt/kWh]
3,National,fuelOil,co2e,3.039000e-04,[mt/kWh]
4,National,naturalGas,so2,9.285943e-10,[mt/kWh]
5,National,naturalGas,nox,1.426940e-07,[mt/kWh]
6,National,naturalGas,pm25,2.830172e-09,[mt/kWh]
7,National,naturalGas,co2e,2.715000e-04,[mt/kWh]
8,National,propane,so2,3.095314e-10,[mt/kWh]
9,National,propane,nox,2.199221e-07,[mt/kWh]


In [11]:
# Create lookup dictionary for fossil fuel emissions factors
emis_fossil_fuel_lookup = {}
fuel_types = df_marg_emis_factors["fuel_type"].unique()

for fuel in fuel_types:
    emis_fossil_fuel_lookup[fuel] = {
        pollutant: df_marg_emis_factors[
            (df_marg_emis_factors["fuel_type"] == fuel) & 
            (df_marg_emis_factors["pollutant"] == pollutant)
        ]["value"].values[0]
        for pollutant in ["co2e", "so2", "nox", "pm25"]
    }

emis_fossil_fuel_lookup


{'fuelOil': {'co2e': 0.0003039,
  'so2': 2.3214857496599997e-09,
  'nox': 2.0119543163720001e-07,
  'pm25': 9.274768578665706e-09},
 'naturalGas': {'co2e': 0.00027150000000000004,
  'so2': 9.285942998640001e-10,
  'nox': 1.42693990745768e-07,
  'pm25': 2.8301719116804617e-09},
 'propane': {'co2e': 0.0002758,
  'so2': 3.09531433288e-10,
  'nox': 2.1992208335112402e-07,
  'pm25': 2.876937828530814e-09}}

### Inflate Marginal Social Cost (Damage) Factors using BLS CPI for All Urban Consumers (CPI-U)
- Series Id:	CUUR0000SA0
- Not Seasonally Adjusted
- Series Title:	All items in U.S. city average, all urban consumers, not seasonally adjusted
- Area:	U.S. city average
- Item:	All items
- Base Period:	1982-84=100

In [12]:
# Load the BLS Inflation Data
filename = 'bls_cpiu_2005-2023.xlsx'
relative_path = os.path.join(r"inflation_data", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# Create a pandas dataframe
df_bls_cpiu = pd.read_excel(file_path, sheet_name='bls_cpiu')

df_bls_cpiu = pd.DataFrame({
    'year': df_bls_cpiu['Year'],
    'cpiu_annual': df_bls_cpiu['Annual']
})

# Obtain the Annual CPIU values for the years of interest
bls_cpi_annual_2008 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2008)].item()
bls_cpi_annual_2010 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2010)].item()
bls_cpi_annual_2013 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2013)].item()
bls_cpi_annual_2018 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2018)].item()
bls_cpi_annual_2019 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2019)].item()
bls_cpi_annual_2020 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2020)].item()
bls_cpi_annual_2021 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2021)].item()
bls_cpi_annual_2022 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2022)].item()
bls_cpi_annual_2023 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2023)].item()

# Precompute constant values
cpi_ratio_2023_2023 = bls_cpi_annual_2023 / bls_cpi_annual_2023
cpi_ratio_2023_2022 = bls_cpi_annual_2023 / bls_cpi_annual_2022
cpi_ratio_2023_2021 = bls_cpi_annual_2023 / bls_cpi_annual_2021  # For EPA VSL (11.3M USD-2021)
cpi_ratio_2023_2020 = bls_cpi_annual_2023 / bls_cpi_annual_2020  # For SCC
cpi_ratio_2023_2019 = bls_cpi_annual_2023 / bls_cpi_annual_2019 
cpi_ratio_2023_2018 = bls_cpi_annual_2023 / bls_cpi_annual_2018 
cpi_ratio_2023_2013 = bls_cpi_annual_2023 / bls_cpi_annual_2013
cpi_ratio_2023_2010 = bls_cpi_annual_2023 / bls_cpi_annual_2010
cpi_ratio_2023_2008 = bls_cpi_annual_2023 / bls_cpi_annual_2008  # For EPA VSL and SCC

Retrieved data for filename: bls_cpiu_2005-2023.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\inflation_data\bls_cpiu_2005-2023.xlsx


### Use the updated Social Cost of Carbon (190 USD-2020/ton CO2) and inflate to USD-2023
- EPA Median for 2% near term discount rate and most commonly mentioned value is 190 USD-2020 using the GIVE model.
- 190 USD-2020 has some inconsistency with the VSL being used. An old study and 2008 VSL is noted
- 190 USD value and inflate to USD 2023 because there is a clear source and ease of replicability.

### Adjustment for VSL
- EASIUR uses a VSL of 8.8M USD-2010 
- New EPA VSL is 11.3M USD-2021
- INFLATE TO $USD-2023

### ALL DOLLAR VALUES ARE NOW IN USD2023, PREVIOUSLY USED $USD-2021

In [13]:
TD_LOSSES = 0.06
TD_LOSSES_MULTIPLIER = 1 / (1 - TD_LOSSES)
EQUIPMENT_SPECS = {'heating': 15, 'waterHeating': 12, 'clothesDrying': 13, 'cooking': 15}
POLLUTANTS = ['so2', 'nox', 'pm25', 'co2e']
EPA_SCC_USD2023_PER_TON = 190 * cpi_ratio_2023_2020 # For co2e adjust SCC

In [14]:
# For CO2 adjust SCC
# Create an adjustment factor for the new Social Cost of Carbon (SCC)
epa_scc = 190 * cpi_ratio_2023_2020
old_scc = 40 * cpi_ratio_2023_2010
scc_adjustment_factor = epa_scc / old_scc

# For Health-Related Emissions Adjust for different Value of a Statistical Life (VSL) values
# Current VSL is $11.3 M USD2021
# INFLATE TO USD2022, PREVIOUSLY USD2021
current_VSL_USD2022 = 11.3 * cpi_ratio_2023_2021

# Easiur uses a VSL of $8.8 M USD2010
# INFLATE TO USD2022, PREVIOUSLY USD2021
easiur_VSL_USD2022 = 8.8 * (cpi_ratio_2023_2010)

# Calculate VSL adjustment factor
vsl_adjustment_factor = current_VSL_USD2022 / easiur_VSL_USD2022

### Quantify monitized HEALTH damages using EASIUR Marginal Social Cost Factors
#### THE STEPS BELOW SUMMARIZE WHAT WAS DONE TO OBTAIN ALL NATIONAL EASIUR VALUES INCLUDED ON GITHUB
- Obtain all of the dwelling unit latitude and longitude values from the metadata columns
- Make a new dataframe of just the longitude and latitude values 
    - Make sure that the order is (longitude, latitude)
    - Do not include the index or column name when exporting 
- Export the CSV
- **Upload csv to EASIUR Website:**
    - Website: https://barney.ce.cmu.edu/~jinhyok/easiur/online/
    - See inputs in respective sections
- Download the file and put it in the 'easiur_batchConversion_download' folder
- Copy and paste the name of the file EASIUR generated when prompted
- Copy and paste the name of the filepath for the 'easiur_batchConversion_download' folder when prompted
- Match up the longitude and latitudes for each dwelling unit with the selected damages

In [15]:
# Create a dataframe containing just the longitude and Latitude
df_EASIUR_batchConversion = pd.DataFrame({
    'Longitude':df_euss_am_baseline['in.weather_file_longitude'],
    'Latitude':df_euss_am_baseline['in.weather_file_latitude'],
})

# Drop duplicate rows based on 'Longitude' and 'Latitude' columns
df_EASIUR_batchConversion.drop_duplicates(subset=['Longitude', 'Latitude'], keep='first', inplace=True)

# Create a location ID for the name of the batch conversion file
while True:
    if menu_state == 'N':
        location_id = 'National'
        print("You chose to analyze all of the United States.")
        break
    elif menu_state == 'Y':
        if menu_city == 'N':
            try:
                location_id = str(input_state)
                print(f"Location ID is: {location_id}")
                break
            except ValueError:
                print("Invalid input for state!")
        elif menu_city == 'Y':
            try:
                location_id = input_cityFilter.replace(', ', '_').strip()
                print(f"Location ID is: {location_id}")
                break
            except AttributeError:
                print("Invalid input for city filter!")
        else:
            print("Incorrect state or city filter assignment!")
    else:
        print("Invalid data location. Check your inputs at the beginning of this notebook!")

# Updated GitHub code has EASIUR file with all unique latitude, longitude coordinates in the US
filename = 'easiur_National2024-06-1421-22.csv'
relative_path = os.path.join(r"margDamages_EASIUR\easiur_batchConversion_download", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_margSocialCosts = pd.read_csv(file_path)

# Convert from kg/MWh to lb/kWh
# Obtain value from the CSV file and convert to lbs pollutant per kWh 

# Define df_marg_social_costs_EASIUR DataFrame first
df_marg_social_costs_EASIUR = pd.DataFrame({
    'Longitude': df_margSocialCosts['Longitude'],
    'Latitude': df_margSocialCosts['Latitude'],
})

# Use df_marg_social_costs_EASIUR in the calculation of other columns
# Also adjust the VSL
df_marg_social_costs_EASIUR['marg_social_costs_pm25'] = round((df_margSocialCosts['PM25 Annual Ground'] * (1/2204.6) * vsl_adjustment_factor), 2)
df_marg_social_costs_EASIUR['marg_social_costs_so2'] = round((df_margSocialCosts['SO2 Annual Ground'] * (1/2204.6) * vsl_adjustment_factor), 2)
df_marg_social_costs_EASIUR['marg_social_costs_nox'] = round((df_margSocialCosts['NOX Annual Ground'] * (1/2204.6) * vsl_adjustment_factor), 2)
df_marg_social_costs_EASIUR['unit'] = '[$USD2023/lb]'

# Dispalay the EASIUR marginal social costs df
df_marg_social_costs_EASIUR

You chose to analyze all of the United States.
Retrieved data for filename: easiur_National2024-06-1421-22.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\margDamages_EASIUR\easiur_batchConversion_download\easiur_National2024-06-1421-22.csv




,Longitude,Latitude,marg_social_costs_pm25,marg_social_costs_so2,marg_social_costs_nox,unit
0,-87.04,31.42,38.00,9.35,1.70,[$USD2023/lb]
1,-85.86,33.59,57.22,10.19,2.64,[$USD2023/lb]
2,-86.39,32.30,52.07,9.70,2.13,[$USD2023/lb]
3,-85.45,31.32,43.54,9.76,1.86,[$USD2023/lb]
4,-86.75,33.56,98.71,11.19,3.00,[$USD2023/lb]
...,...,...,...,...,...,...
1024,-106.72,44.38,10.33,5.02,0.76,[$USD2023/lb]
1025,-105.54,44.34,11.44,6.05,0.96,[$USD2023/lb]
1026,-107.95,43.97,8.85,4.47,0.61,[$USD2023/lb]
1027,-108.08,44.52,9.29,5.74,0.82,[$USD2023/lb]


In [16]:
# FOSSIL FUELS DAMAGES LOOKUP
# Create a damages_fossil_fuel_lookup dictionary from df_marg_social_costs_EASIUR
damages_fossil_fuel_lookup = df_marg_social_costs_EASIUR.groupby(['Longitude', 'Latitude']).first().to_dict()
damages_fossil_fuel_lookup

{'marg_social_costs_pm25': {(-124.25, 43.42): 39.96,
  (-124.24, 41.78): 26.13,
  (-124.11, 40.98): 25.91,
  (-124.07, 44.62): 39.38,
  (-123.94, 46.97): 60.55,
  (-123.88, 46.16): 51.33,
  (-123.5, 48.12): 61.62,
  (-123.38, 42.62): 34.41,
  (-123.36, 43.24): 32.46,
  (-123.29, 44.5): 72.79,
  (-123.21, 44.13): 70.7,
  (-123.2, 39.13): 38.57,
  (-123.15, 47.24): 70.68,
  (-123.13, 45.2): 124.73,
  (-123.02, 48.52): 66.41,
  (-123.0, 44.91): 112.52,
  (-122.95, 45.54): 124.73,
  (-122.9, 46.97): 135.35,
  (-122.87, 42.39): 38.26,
  (-122.86, 45.77): 130.55,
  (-122.81, 38.51): 80.93,
  (-122.77, 45.25): 94.07,
  (-122.76, 47.49): 117.71,
  (-122.66, 45.62): 130.55,
  (-122.6, 45.59): 206.04,
  (-122.55, 47.12): 185.76,
  (-122.54, 48.79): 61.79,
  (-122.47, 41.78): 19.69,
  (-122.43, 47.67): 191.74,
  (-122.4, 37.62): 301.86,
  (-122.31, 40.52): 37.49,
  (-122.28, 38.21): 157.85,
  (-122.28, 47.91): 173.1,
  (-122.25, 40.15): 31.58,
  (-122.21, 47.49): 179.48,
  (-122.15, 48.16): 93.42

## Emissions from Electricity Generation

### Climate-Related Emissions from CAMBIUM LRMER/SRMER 
### Includes pre-combustion (fugitive) and combustion

In [17]:
# INTERPOLATE ANNUAL DATA BETWEEN 5-YEAR TIME STEPS
print("""
-------------------------------------------------------------------------------------------------------
CLIMATE DAMAGES FROM CAMBIUM
-------------------------------------------------------------------------------------------------------
- Load CSV
- Convert MWh --> kWh and kg --> metric tons (mt)
- Inflate updated Social Cost of Carbon from $190 USD2020 to $USD2023
- Convert SCC to $USD2023/lb
- Calculate damage factors for CO2e: LRMER[lb/kWh] * SCC[$USD2023/lb] = $USD2023/kWh
- Map state, county pairs to Cambium 2023 GEA region

Possibly for other emissions projections:
- Calculate projection factors for each group of scenario and GEA region for 2025 to 2050 (normalize all annual data in group by 2025 value) 
-------------------------------------------------------------------------------------------------------
""")


-------------------------------------------------------------------------------------------------------
CLIMATE DAMAGES FROM CAMBIUM
-------------------------------------------------------------------------------------------------------
- Load CSV
- Convert MWh --> kWh and kg --> metric tons (mt)
- Inflate updated Social Cost of Carbon from $190 USD2020 to $USD2023
- Convert SCC to $USD2023/lb
- Calculate damage factors for CO2e: LRMER[lb/kWh] * SCC[$USD2023/lb] = $USD2023/kWh
- Map state, county pairs to Cambium 2023 GEA region

Possibly for other emissions projections:
- Calculate projection factors for each group of scenario and GEA region for 2025 to 2050 (normalize all annual data in group by 2025 value) 
-------------------------------------------------------------------------------------------------------



In [18]:
import os
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

print("""
-------------------------------------------------------------------------------------------------------
PRE-IRA LONG RUN AND SHORT RUN MARGINAL EMISSIONS RATES (LRMER, SRMER) FROM CAMBIUM 2021 RELEASE
-------------------------------------------------------------------------------------------------------
""")

# CAMBIUM 2021 FOR PRE-IRA SCENARIO
filename = 'cambium21_midCase_annual_gea.xlsx'
relative_path = os.path.join(r"projections", filename)
file_path = os.path.join(project_root, relative_path)
df_cambium21_margEmis_electricity = pd.read_excel(io=file_path, sheet_name='proc_Cambium21_MidCase_gea')

print(f"""
Retrieved data for filename: {filename}
Located at filepath: {file_path}

Loading dataframe ...
Creating lookup dictionary for LRMER and SRMER ...
-------------------------------------------------------------------------------------------------------
""")

# Calculate electricity emission factors for Cambium 2021
# Process the data using the provided function to interpolate and convert units
df_cambium21_processed = calculate_electricity_co2e_cambium(df_cambium21_margEmis_electricity)

# # Display the processed DataFrame
# df_cambium21_processed

# Create the lookup dictionary using the create_cambium_emission_factor_lookup function
emis_preIRA_co2e_cambium21_lookup = create_cambium_co2e_lookup(df_cambium21_processed)

# Display the lookup dictionary
emis_preIRA_co2e_cambium21_lookup


-------------------------------------------------------------------------------------------------------
PRE-IRA LONG RUN AND SHORT RUN MARGINAL EMISSIONS RATES (LRMER, SRMER) FROM CAMBIUM 2021 RELEASE
-------------------------------------------------------------------------------------------------------


Retrieved data for filename: cambium21_midCase_annual_gea.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\projections\cambium21_midCase_annual_gea.xlsx

Loading dataframe ...
Creating lookup dictionary for LRMER and SRMER ...
-------------------------------------------------------------------------------------------------------



{('MidCase',
  'AZNMc'): {2022: {'lrmer_ton_per_kWh_co2e': 0.0005256,
   'srmer_ton_per_kWh_co2e': 0.0008118}, 2023: {'lrmer_ton_per_kWh_co2e': 0.000507,
   'srmer_ton_per_kWh_co2e': 0.0008100999999999999}, 2024: {'lrmer_ton_per_kWh_co2e': 0.0004883999999999999,
   'srmer_ton_per_kWh_co2e': 0.0008084}, 2025: {'lrmer_ton_per_kWh_co2e': 0.00045575,
   'srmer_ton_per_kWh_co2e': 0.0008122}, 2026: {'lrmer_ton_per_kWh_co2e': 0.00042310000000000004,
   'srmer_ton_per_kWh_co2e': 0.000816}, 2027: {'lrmer_ton_per_kWh_co2e': 0.00036945000000000006,
   'srmer_ton_per_kWh_co2e': 0.00080035}, 2028: {'lrmer_ton_per_kWh_co2e': 0.00031580000000000003,
   'srmer_ton_per_kWh_co2e': 0.0007847}, 2029: {'lrmer_ton_per_kWh_co2e': 0.00031470000000000006,
   'srmer_ton_per_kWh_co2e': 0.00076725}, 2030: {'lrmer_ton_per_kWh_co2e': 0.00031360000000000003,
   'srmer_ton_per_kWh_co2e': 0.0007497999999999999}, 2031: {'lrmer_ton_per_kWh_co2e': 0.00029605,
   'srmer_ton_per_kWh_co2e': 0.00073815}, 2032: {'lrmer_ton_pe

In [19]:
import os
import pandas as pd

print("""
-------------------------------------------------------------------------------------------------------
IRA LONG RUN AND SHORT RUN MARGINAL EMISSIONS RATES (LRMER, SRMER) FROM CAMBIUM 2022 RELEASE
-------------------------------------------------------------------------------------------------------
""")

# CAMBIUM 2022 FOR IRA SCENARIO
filename = 'cambium22_allScenarios_annual_gea.xlsx'
relative_path = os.path.join(r"projections", filename)
file_path = os.path.join(project_root, relative_path)
df_cambium22_margEmis_electricity = pd.read_excel(io=file_path, sheet_name='proc_Cambium22_MidCase_gea')

print(f"""
Retrieved data for filename: {filename}
Located at filepath: {file_path}

Loading dataframe ...
Creating lookup dictionary for 2024 LRMER and SRMER ...
-------------------------------------------------------------------------------------------------------
""")

# Calculate electricity emission factors for Cambium 2021
# Process the data using the provided function to interpolate and convert units
df_cambium22_processed = calculate_electricity_co2e_cambium(df_cambium22_margEmis_electricity)

# # Display the processed DataFrame
# df_cambium22_processed

# Create the lookup dictionary using the create_cambium_co2e_lookup function
emis_IRA_co2e_cambium22_lookup = create_cambium_co2e_lookup(df_cambium22_processed)

# Display the lookup dictionary
emis_IRA_co2e_cambium22_lookup


-------------------------------------------------------------------------------------------------------
IRA LONG RUN AND SHORT RUN MARGINAL EMISSIONS RATES (LRMER, SRMER) FROM CAMBIUM 2022 RELEASE
-------------------------------------------------------------------------------------------------------


Retrieved data for filename: cambium22_allScenarios_annual_gea.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\projections\cambium22_allScenarios_annual_gea.xlsx

Loading dataframe ...
Creating lookup dictionary for 2024 LRMER and SRMER ...
-------------------------------------------------------------------------------------------------------



{('MidCase',
  'AZNMc'): {2024: {'lrmer_ton_per_kWh_co2e': 0.0003269,
   'srmer_ton_per_kWh_co2e': 0.0007771}, 2025: {'lrmer_ton_per_kWh_co2e': 0.0002604,
   'srmer_ton_per_kWh_co2e': 0.0007434000000000001}, 2026: {'lrmer_ton_per_kWh_co2e': 0.0001939,
   'srmer_ton_per_kWh_co2e': 0.0007097000000000001}, 2027: {'lrmer_ton_per_kWh_co2e': 0.00016219999999999999,
   'srmer_ton_per_kWh_co2e': 0.0006430499999999999}, 2028: {'lrmer_ton_per_kWh_co2e': 0.0001305,
   'srmer_ton_per_kWh_co2e': 0.0005764}, 2029: {'lrmer_ton_per_kWh_co2e': 0.0001171,
   'srmer_ton_per_kWh_co2e': 0.00053155}, 2030: {'lrmer_ton_per_kWh_co2e': 0.0001037,
   'srmer_ton_per_kWh_co2e': 0.0004867}, 2031: {'lrmer_ton_per_kWh_co2e': 0.00011284,
   'srmer_ton_per_kWh_co2e': 0.00046903999999999994}, 2032: {'lrmer_ton_per_kWh_co2e': 0.00012198000000000001,
   'srmer_ton_per_kWh_co2e': 0.00045138}, 2033: {'lrmer_ton_per_kWh_co2e': 0.00013112,
   'srmer_ton_per_kWh_co2e': 0.00043371999999999996}, 2034: {'lrmer_ton_per_kWh_co2e':

### Use the updated Social Cost of Carbon (190 USD-2020/ton co2e) and inflate to USD-2023
- EPA Median for 2% near term discount rate and most commonly mentioned value is 190 USD-2020 using the GIVE model.
- 190 USD-2020 has some inconsistency with the VSL being used. An old study and 2008 VSL is noted
- 190 USD value and inflate to USD 2023 because there is a clear source and ease of replicability.

In [20]:
# For co2e adjust SCC
EPA_SCC_USD2023_PER_TON = 190 * cpi_ratio_2023_2020

print(f"""
Steps 3 and 4: Obtain BLS CPI-U Data and Inflate Current Social Cost of Carbon (SCC) to USD2023
      
EPA Median for 2% near term discount rate and most commonly mentioned value is 190 USD-2020 using the GIVE model.
Inflate 190 $USD-2020 Social Cost of Carbon to $USD-2023

SCC Value used in analysis is: ${round(EPA_SCC_USD2023_PER_TON, 2)} per mt CO2e
""")


Steps 3 and 4: Obtain BLS CPI-U Data and Inflate Current Social Cost of Carbon (SCC) to USD2023
      
EPA Median for 2% near term discount rate and most commonly mentioned value is 190 USD-2020 using the GIVE model.
Inflate 190 $USD-2020 Social Cost of Carbon to $USD-2023

SCC Value used in analysis is: $223.69 per mt CO2e



## HEALTH-RELATED EMISSIONS: Grid Emissions Intensity Projections

### PROJECTION FACTORS FOR FUTURE GRID EMISSIONS INTENSITY (Coal Generation Reduction)

In [21]:
import os
import pandas as pd

print("""
-------------------------------------------------------------------------------------------------------
COAL USED IN ELECTRICITY GENERATION FROM CAMBIUM 2021 RELEASE
-------------------------------------------------------------------------------------------------------
""")

# CAMBIUM 2022 FOR IRA SCENARIO
filename = 'cambium21_midCase_annual_gea.xlsx'
relative_path = os.path.join(r"projections", filename)
file_path = os.path.join(project_root, relative_path)
df_cambium21_COAL_processed = pd.read_excel(io=file_path, sheet_name='proc_Cambium21_coal_gea')

print(f"""
Retrieved data for filename: {filename}
Located at filepath: {file_path}
-------------------------------------------------------------------------------------------------------
""")
# df_cambium21_COAL_processed 


-------------------------------------------------------------------------------------------------------
COAL USED IN ELECTRICITY GENERATION FROM CAMBIUM 2021 RELEASE
-------------------------------------------------------------------------------------------------------


Retrieved data for filename: cambium21_midCase_annual_gea.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\projections\cambium21_midCase_annual_gea.xlsx
-------------------------------------------------------------------------------------------------------



In [22]:
import os
import pandas as pd

print("""
-------------------------------------------------------------------------------------------------------
COAL USED IN ELECTRICITY GENERATION FROM CAMBIUM 2022 RELEASE
-------------------------------------------------------------------------------------------------------
""")

# CAMBIUM 2022 FOR IRA SCENARIO
filename = 'cambium22_allScenarios_annual_gea.xlsx'
relative_path = os.path.join(r"projections", filename)
file_path = os.path.join(project_root, relative_path)
df_cambium22_COAL_processed = pd.read_excel(io=file_path, sheet_name='proc_Cambium22_coal_gea')

print(f"""
Retrieved data for filename: {filename}
Located at filepath: {file_path}
-------------------------------------------------------------------------------------------------------
""")
# df_cambium22_COAL_processed 


-------------------------------------------------------------------------------------------------------
COAL USED IN ELECTRICITY GENERATION FROM CAMBIUM 2022 RELEASE
-------------------------------------------------------------------------------------------------------


Retrieved data for filename: cambium22_allScenarios_annual_gea.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\projections\cambium22_allScenarios_annual_gea.xlsx
-------------------------------------------------------------------------------------------------------



In [23]:
mapping = {
    'AKGD': None,       # Alaska Grid - Not included
    'AKMS': None,       # Alaska Miscellaneous - Not included
    'AZNM': 'AZNMc',    # Arizona/New Mexico Power Area
    'CAMX': 'CAMXc',    # California Mexico
    'ERCT': 'ERCTc',    # Electric Reliability Council of Texas
    'FRCC': 'FRCCc',    # Florida Reliability Coordinating Council
    'HIMS': None,       # Hawaii Maui Subregion - Not included
    'HIOA': None,       # Hawaii Oahu Subregion - Not included
    'MROE': 'MROEc',    # Midwest Reliability Organization East
    'MROW': 'MROWc',    # Midwest Reliability Organization West
    'NEWE': 'NEWEc',    # New England
    'NWPP': 'NWPPc',    # Northwest Power Pool
    'NYCW': 'NYSTc',    # New York City/Westchester mapped to New York State
    'NYLI': 'NYSTc',    # New York Long Island mapped to New York State
    'NYUP': 'NYSTc',    # New York Upstate mapped to New York State
    'PRMS': None,       # Puerto Rico Miscellaneous - Not included
    'RFCE': 'RFCEc',    # ReliabilityFirst Corporation East
    'RFCM': 'RFCMc',    # ReliabilityFirst Corporation Midwest
    'RFCW': 'RFCWc',    # ReliabilityFirst Corporation West
    'RMPA': 'RMPAc',    # Rocky Mountain Power Area
    'SPNO': 'SPNOc',    # Southwest Power Pool North
    'SPSO': 'SPSOc',    # Southwest Power Pool South
    'SRMV': 'SRMVc',    # SERC Reliability Corporation Mississippi Valley
    'SRMW': 'SRMWc',    # SERC Reliability Corporation Midwest
    'SRSO': 'SRSOc',    # SERC Reliability Corporation South
    'SRTV': 'SRTVc',    # SERC Reliability Corporation Tennessee Valley
    'SRVC': 'SRVCc',    # SERC Reliability Corporation Virginia/Carolina
}

# EPA eGRID Coal Generation Data for 2018 to 2022 ()
filename = 'epa_eGRID_total_coal_generation_2018_2022.xlsx'
relative_path = os.path.join(r"projections", filename)
file_path = os.path.join(project_root, relative_path)
df_epa_eGRID_COAL_processed = pd.read_excel(io=file_path, sheet_name='coal_generation_2018_2022')

print(f"""
Retrieved data for filename: {filename}
Located at filepath: {file_path}
-------------------------------------------------------------------------------------------------------
""") 

# Apply the mapping to the 'cambium_gea_region' column
df_epa_eGRID_COAL_processed['cambium_gea_region'] = df_epa_eGRID_COAL_processed['eGRID_subregion'].map(mapping)

# Drop rows where 'cambium_gea_region' is None (regions not included in the mapping)
df_epa_eGRID_COAL_processed = df_epa_eGRID_COAL_processed.dropna(subset=['cambium_gea_region'])

# Group by 'cambium_gea_region' and aggregate
df_epa_eGRID_COAL_processed = (
    df_epa_eGRID_COAL_processed
    .groupby('cambium_gea_region', as_index=False)
    .agg({
        'eGRID_subregion': 'first',  # Retain the first value (or use a different strategy)
        'coal_MWh_2018': 'sum',
        'coal_MWh_2019': 'sum',
        'coal_MWh_2020': 'sum',
        'coal_MWh_2021': 'sum',
        'coal_MWh_2022': 'sum'
    })
)

# Step 1: Melt the DataFrame to Long Format
columns_to_melt = ['coal_MWh_2018', 'coal_MWh_2019', 'coal_MWh_2020', 'coal_MWh_2021', 'coal_MWh_2022']
df_melted = pd.melt(
    df_epa_eGRID_COAL_processed,
    id_vars=['eGRID_subregion', 'cambium_gea_region'],
    value_vars=columns_to_melt,
    var_name='year',
    value_name='coal_MWh'
)

# Step 2: Extract the Year from Column Names
df_melted['year'] = df_melted['year'].str.extract('(\d{4})').astype(int)


Retrieved data for filename: epa_eGRID_total_coal_generation_2018_2022.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\projections\epa_eGRID_total_coal_generation_2018_2022.xlsx
-------------------------------------------------------------------------------------------------------



In [24]:
# Pre-IRA Scenario
# Step 3: Create Columns to Match the Target DataFrame
df_preIRA_transformed = pd.DataFrame({
    'data_source': 'EPA_eGRID',
    'scenario': 'MidCase',
    'eGRID_subregion': df_melted['eGRID_subregion'],
    'gea_region': df_melted['cambium_gea_region'],
    'year': df_melted['year'],
    'coal_MWh': df_melted['coal_MWh'],
})

# Step 4: Combine the DataFrames
df_preIRA_coal_generation = pd.concat([df_preIRA_transformed, df_cambium21_COAL_processed], ignore_index=True)
# df_preIRA_coal_generation

# Calculate Coal Generation Projection Factors
df_preIRA_coal_projection_factors = calculate_coal_projection_factors(df_preIRA_coal_generation)
df_preIRA_coal_projection_factors

,scenario,gea_region,year,coal_MWh,coal_projection_factors
0,MidCase,AZNMc,2018,44146154.50,1.000000
1,MidCase,AZNMc,2019,37909639.38,0.858730
2,MidCase,AZNMc,2020,26535437.00,0.601082
3,MidCase,AZNMc,2021,26836891.44,0.607910
4,MidCase,AZNMc,2022,26741465.00,0.605748
...,...,...,...,...,...
655,MidCase,SRVCc,2046,27152428.00,0.433227
656,MidCase,SRVCc,2047,27506921.00,0.438883
657,MidCase,SRVCc,2048,27861414.00,0.444539
658,MidCase,SRVCc,2049,21319390.00,0.340159


In [25]:
# IRA Reference Scenario
# Step 3: Create Columns to Match the Target DataFrame
df_iraRef_transformed = pd.DataFrame({
    'data_source': 'EPA_eGRID',
    'scenario': 'MidCase',
    'eGRID_subregion': df_melted['eGRID_subregion'],
    'gea_region': df_melted['cambium_gea_region'],
    'year': df_melted['year'],
    'coal_MWh': df_melted['coal_MWh'],
})

# Step 4: Combine the DataFrames
df_iraRef_coal_generation = pd.concat([df_iraRef_transformed, df_cambium22_COAL_processed], ignore_index=True)
# df_iraRef_coal_generation

# Calculate Coal Generation Projection Factors
df_iraRef_coal_projection_factors = calculate_coal_projection_factors(df_iraRef_coal_generation)
df_iraRef_coal_projection_factors

,scenario,gea_region,year,coal_MWh,coal_projection_factors
0,MidCase,AZNMc,2018,44146154.50,1.000000
1,MidCase,AZNMc,2019,37909639.38,0.858730
2,MidCase,AZNMc,2020,26535437.00,0.601082
3,MidCase,AZNMc,2021,26836891.44,0.607910
4,MidCase,AZNMc,2022,26741465.00,0.605748
...,...,...,...,...,...
655,MidCase,SRVCc,2046,3826239.64,0.061049
656,MidCase,SRVCc,2047,3350872.78,0.053464
657,MidCase,SRVCc,2048,2875505.92,0.045880
658,MidCase,SRVCc,2049,2400139.06,0.038295


## ADJUSTED Electricity CEDM-EASIUR Marginal Damages: Pre-IRA and IRA-Reference
- Factor Type: Marginal
- Calculation Method: Regression
- Metric: Marginal Damages EASIUR [USD per MWh or kWh]
- Year: 2018
- Regional Aggregation: eGRID subregion (all regions)
- Pollutants: SO2, NOx, PM2.5 CO2

SCC Adjustment: We use the EPA suggested 190 USD-2020 value for the social cost of carbon and inflate to 2022 USD. **PREVIOUSLY USED 2021 USD**

VSL: "We use a value of a statistical life (VSL) of USD 8.8 million (in 2010 dollars) for both our AP2 and EASIUR calculations. EASIUR reports damage intensities in USD/metric ton using this VSL and dollar year."

In [26]:
filename = 'Generation-MARREG-DAMEASIUR-egrid-byYear_health2018.csv'
relative_path = os.path.join(r"margDamages_EASIUR", filename)
file_path = os.path.join(project_root, relative_path)
df_margDamages_health2018 = pd.read_csv(file_path, index_col=0)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# Marginal damages [$/kWh]
# Inflate from 2010 to 2022
# Note only 2018 data available, used in place of 2021
df_margDamages_EASIUR_health = pd.DataFrame({
    'subregion_eGRID': df_margDamages_health2018['region'],
    'pollutant': df_margDamages_health2018['pollutant'],
    'unit': '[$USD2023/kWh]',
    'margDamages_dollarPerkWh_adjustVSL_ref': (df_margDamages_health2018['factor'] * (vsl_adjustment_factor) * (1/1000)) * (cpi_ratio_2023_2010),
    'margDamages_dollarPerkWh_adjustVSL_2018': (df_margDamages_health2018['factor'] * (vsl_adjustment_factor) * (1/1000)) * (cpi_ratio_2023_2010)
})

# Apply the mapping to the 'cambium_gea_region' column
df_margDamages_EASIUR_health['cambium_gea_region'] = df_margDamages_EASIUR_health['subregion_eGRID'].map(mapping)
df_margDamages_EASIUR_health

Retrieved data for filename: Generation-MARREG-DAMEASIUR-egrid-byYear_health2018.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\margDamages_EASIUR\Generation-MARREG-DAMEASIUR-egrid-byYear_health2018.csv


,subregion_eGRID,pollutant,unit,margDamages_dollarPerkWh_adjustVSL_ref,margDamages_dollarPerkWh_adjustVSL_2018,cambium_gea_region
1,AZNM,so2,[$USD2023/kWh],0.001760,0.001760,AZNMc
2,AZNM,nox,[$USD2023/kWh],0.000575,0.000575,AZNMc
3,AZNM,pm25,[$USD2023/kWh],0.002026,0.002026,AZNMc
4,CAMX,so2,[$USD2023/kWh],0.000599,0.000599,CAMXc
5,CAMX,nox,[$USD2023/kWh],0.000714,0.000714,CAMXc
...,...,...,...,...,...,...
62,SRTV,nox,[$USD2023/kWh],0.002780,0.002780,SRTVc
63,SRTV,pm25,[$USD2023/kWh],0.007428,0.007428,SRTVc
64,SRVC,so2,[$USD2023/kWh],0.008226,0.008226,SRVCc
65,SRVC,nox,[$USD2023/kWh],0.002431,0.002431,SRVCc


### Pre-IRA Projections

In [27]:
# Create dictionaries mapping 'gea_region' to marginal damage factors for each pollutant
# For SO2
so2_marg_damages = df_margDamages_EASIUR_health[df_margDamages_EASIUR_health['pollutant'] == 'so2'] \
    .set_index('cambium_gea_region')['margDamages_dollarPerkWh_adjustVSL_ref'].to_dict()

# For NOx
nox_marg_damages = df_margDamages_EASIUR_health[df_margDamages_EASIUR_health['pollutant'] == 'nox'] \
    .set_index('cambium_gea_region')['margDamages_dollarPerkWh_adjustVSL_ref'].to_dict()

# For PM2.5
pm25_marg_damages = df_margDamages_EASIUR_health[df_margDamages_EASIUR_health['pollutant'] == 'pm25'] \
    .set_index('cambium_gea_region')['margDamages_dollarPerkWh_adjustVSL_ref'].to_dict()

# Map the marginal damage factors to the projection factors dataframe
df_preIRA_coal_projection_factors['so2_marg_damage'] = df_preIRA_coal_projection_factors['gea_region'].map(so2_marg_damages)
df_preIRA_coal_projection_factors['nox_marg_damage'] = df_preIRA_coal_projection_factors['gea_region'].map(nox_marg_damages)
df_preIRA_coal_projection_factors['pm25_marg_damage'] = df_preIRA_coal_projection_factors['gea_region'].map(pm25_marg_damages)

# Calculate the new columns by multiplying the coal projection factors with the marginal damages
df_preIRA_coal_projection_factors['so2_dollarPerkWh_adjustVSL'] = (
    df_preIRA_coal_projection_factors['coal_projection_factors'] * df_preIRA_coal_projection_factors['so2_marg_damage']
)
df_preIRA_coal_projection_factors['nox_dollarPerkWh_adjustVSL'] = (
    df_preIRA_coal_projection_factors['coal_projection_factors'] * df_preIRA_coal_projection_factors['nox_marg_damage']
)
df_preIRA_coal_projection_factors['pm25_dollarPerkWh_adjustVSL'] = (
    df_preIRA_coal_projection_factors['coal_projection_factors'] * df_preIRA_coal_projection_factors['pm25_marg_damage']
)

# Optionally, drop the intermediate marginal damage columns if they're no longer needed
df_preIRA_coal_projection_factors.drop(
    columns=['so2_marg_damage', 'nox_marg_damage', 'pm25_marg_damage'], 
    inplace=True
)
df_preIRA_coal_projection_factors

,scenario,gea_region,year,coal_MWh,coal_projection_factors,so2_dollarPerkWh_adjustVSL,nox_dollarPerkWh_adjustVSL,pm25_dollarPerkWh_adjustVSL
0,MidCase,AZNMc,2018,44146154.50,1.000000,0.001760,0.000575,0.002026
1,MidCase,AZNMc,2019,37909639.38,0.858730,0.001512,0.000494,0.001740
2,MidCase,AZNMc,2020,26535437.00,0.601082,0.001058,0.000346,0.001218
3,MidCase,AZNMc,2021,26836891.44,0.607910,0.001070,0.000350,0.001232
4,MidCase,AZNMc,2022,26741465.00,0.605748,0.001066,0.000348,0.001227
...,...,...,...,...,...,...,...,...
655,MidCase,SRVCc,2046,27152428.00,0.433227,0.003564,0.001053,0.003264
656,MidCase,SRVCc,2047,27506921.00,0.438883,0.003610,0.001067,0.003307
657,MidCase,SRVCc,2048,27861414.00,0.444539,0.003657,0.001081,0.003349
658,MidCase,SRVCc,2049,21319390.00,0.340159,0.002798,0.000827,0.002563


In [28]:
# Initialize the lookup dictionary
damages_preIRA_health_damages_lookup = {}

# Group the dataframe
grouped = df_preIRA_coal_projection_factors.groupby(['scenario', 'gea_region'])

for (scenario, gea_region), group in grouped:
    year_dict = {}
    for _, row in group.iterrows():
        year = int(row['year'])
        data_dict = {
            'so2_dollarPerkWh_adjustVSL': row['so2_dollarPerkWh_adjustVSL'],
            'nox_dollarPerkWh_adjustVSL': row['nox_dollarPerkWh_adjustVSL'],
            'pm25_dollarPerkWh_adjustVSL': row['pm25_dollarPerkWh_adjustVSL']
        }
        year_dict[year] = data_dict
    damages_preIRA_health_damages_lookup[(scenario, gea_region)] = year_dict

# Now, lookup_dict contains the desired nested dictionary
damages_preIRA_health_damages_lookup

{('MidCase',
  'AZNMc'): {2018: {'so2_dollarPerkWh_adjustVSL': 0.0017604045476692913,
   'nox_dollarPerkWh_adjustVSL': 0.000575015374879764,
   'pm25_dollarPerkWh_adjustVSL': 0.0020259847537919245}, 2019: {'so2_dollarPerkWh_adjustVSL': 0.0015117126807739248,
   'nox_dollarPerkWh_adjustVSL': 0.0004937831108176674,
   'pm25_dollarPerkWh_adjustVSL': 0.0017397744441280825}, 2020: {'so2_dollarPerkWh_adjustVSL': 0.0010581466154473767,
   'nox_dollarPerkWh_adjustVSL': 0.00034563110710250785,
   'pm25_dollarPerkWh_adjustVSL': 0.001217781965521054}, 2021: {'so2_dollarPerkWh_adjustVSL': 0.0010701676345622147,
   'nox_dollarPerkWh_adjustVSL': 0.0003495576311630751,
   'pm25_dollarPerkWh_adjustVSL': 0.0012316165136559971}, 2022: {'so2_dollarPerkWh_adjustVSL': 0.0010663623396085199,
   'nox_dollarPerkWh_adjustVSL': 0.00034831467646426797,
   'pm25_dollarPerkWh_adjustVSL': 0.0012272371398523595}, 2023: {'so2_dollarPerkWh_adjustVSL': 0.0012037391306330793,
   'nox_dollarPerkWh_adjustVSL': 0.000393187

### IRA-Reference Projections

In [29]:
# Create dictionaries mapping 'gea_region' to marginal damage factors for each pollutant
# For SO2
so2_marg_damages = df_margDamages_EASIUR_health[df_margDamages_EASIUR_health['pollutant'] == 'so2'] \
    .set_index('cambium_gea_region')['margDamages_dollarPerkWh_adjustVSL_ref'].to_dict()

# For NOx
nox_marg_damages = df_margDamages_EASIUR_health[df_margDamages_EASIUR_health['pollutant'] == 'nox'] \
    .set_index('cambium_gea_region')['margDamages_dollarPerkWh_adjustVSL_ref'].to_dict()

# For PM2.5
pm25_marg_damages = df_margDamages_EASIUR_health[df_margDamages_EASIUR_health['pollutant'] == 'pm25'] \
    .set_index('cambium_gea_region')['margDamages_dollarPerkWh_adjustVSL_ref'].to_dict()

# Map the marginal damage factors to the projection factors dataframe
df_iraRef_coal_projection_factors['so2_marg_damage'] = df_iraRef_coal_projection_factors['gea_region'].map(so2_marg_damages)
df_iraRef_coal_projection_factors['nox_marg_damage'] = df_iraRef_coal_projection_factors['gea_region'].map(nox_marg_damages)
df_iraRef_coal_projection_factors['pm25_marg_damage'] = df_iraRef_coal_projection_factors['gea_region'].map(pm25_marg_damages)

# Calculate the new columns by multiplying the coal projection factors with the marginal damages
df_iraRef_coal_projection_factors['so2_dollarPerkWh_adjustVSL'] = (
    df_iraRef_coal_projection_factors['coal_projection_factors'] * df_iraRef_coal_projection_factors['so2_marg_damage']
)
df_iraRef_coal_projection_factors['nox_dollarPerkWh_adjustVSL'] = (
    df_iraRef_coal_projection_factors['coal_projection_factors'] * df_iraRef_coal_projection_factors['nox_marg_damage']
)
df_iraRef_coal_projection_factors['pm25_dollarPerkWh_adjustVSL'] = (
    df_iraRef_coal_projection_factors['coal_projection_factors'] * df_iraRef_coal_projection_factors['pm25_marg_damage']
)

# Optionally, drop the intermediate marginal damage columns if they're no longer needed
df_iraRef_coal_projection_factors.drop(
    columns=['so2_marg_damage', 'nox_marg_damage', 'pm25_marg_damage'], 
    inplace=True
)
df_iraRef_coal_projection_factors

,scenario,gea_region,year,coal_MWh,coal_projection_factors,so2_dollarPerkWh_adjustVSL,nox_dollarPerkWh_adjustVSL,pm25_dollarPerkWh_adjustVSL
0,MidCase,AZNMc,2018,44146154.50,1.000000,0.001760,0.000575,0.002026
1,MidCase,AZNMc,2019,37909639.38,0.858730,0.001512,0.000494,0.001740
2,MidCase,AZNMc,2020,26535437.00,0.601082,0.001058,0.000346,0.001218
3,MidCase,AZNMc,2021,26836891.44,0.607910,0.001070,0.000350,0.001232
4,MidCase,AZNMc,2022,26741465.00,0.605748,0.001066,0.000348,0.001227
...,...,...,...,...,...,...,...,...
655,MidCase,SRVCc,2046,3826239.64,0.061049,0.000502,0.000148,0.000460
656,MidCase,SRVCc,2047,3350872.78,0.053464,0.000440,0.000130,0.000403
657,MidCase,SRVCc,2048,2875505.92,0.045880,0.000377,0.000112,0.000346
658,MidCase,SRVCc,2049,2400139.06,0.038295,0.000315,0.000093,0.000289


In [30]:
# Initialize the lookup dictionary
damages_iraRef_health_damages_lookup = {}

# Group the dataframe
grouped = df_iraRef_coal_projection_factors.groupby(['scenario', 'gea_region'])

for (scenario, gea_region), group in grouped:
    year_dict = {}
    for _, row in group.iterrows():
        year = int(row['year'])
        data_dict = {
            'so2_dollarPerkWh_adjustVSL': row['so2_dollarPerkWh_adjustVSL'],
            'nox_dollarPerkWh_adjustVSL': row['nox_dollarPerkWh_adjustVSL'],
            'pm25_dollarPerkWh_adjustVSL': row['pm25_dollarPerkWh_adjustVSL']
        }
        year_dict[year] = data_dict
    damages_iraRef_health_damages_lookup[(scenario, gea_region)] = year_dict

# Now, lookup_dict contains the desired nested dictionary
damages_iraRef_health_damages_lookup

{('MidCase',
  'AZNMc'): {2018: {'so2_dollarPerkWh_adjustVSL': 0.0017604045476692913,
   'nox_dollarPerkWh_adjustVSL': 0.000575015374879764,
   'pm25_dollarPerkWh_adjustVSL': 0.0020259847537919245}, 2019: {'so2_dollarPerkWh_adjustVSL': 0.0015117126807739248,
   'nox_dollarPerkWh_adjustVSL': 0.0004937831108176674,
   'pm25_dollarPerkWh_adjustVSL': 0.0017397744441280825}, 2020: {'so2_dollarPerkWh_adjustVSL': 0.0010581466154473767,
   'nox_dollarPerkWh_adjustVSL': 0.00034563110710250785,
   'pm25_dollarPerkWh_adjustVSL': 0.001217781965521054}, 2021: {'so2_dollarPerkWh_adjustVSL': 0.0010701676345622147,
   'nox_dollarPerkWh_adjustVSL': 0.0003495576311630751,
   'pm25_dollarPerkWh_adjustVSL': 0.0012316165136559971}, 2022: {'so2_dollarPerkWh_adjustVSL': 0.0010663623396085199,
   'nox_dollarPerkWh_adjustVSL': 0.00034831467646426797,
   'pm25_dollarPerkWh_adjustVSL': 0.0012272371398523595}, 2023: {'so2_dollarPerkWh_adjustVSL': 0.0009887543357298897,
   'nox_dollarPerkWh_adjustVSL': 0.000322964

### Calculate End-use specific marginal damages
**I used the total emissions column for each of the end uses for the following reasons:**
- Most homes only have 1 of each end-use, so it is unlikely that the homes have a significant consumption values from different fuel types. Thus, the total consumption and total emissions column (sum of each dwelling units consumption by end-use for each fuel) is fine to use to calculate marginal damages (social cost)
- We can visualize the emissions in 2 by 2 grid (CO2, PM25, SO2, NOx) with each appliance's heating fuel in a different shape or color. 

### Baseline Marginal Damages: WHOLE-HOME

In [31]:
print("""
-------------------------------------------------------------------------------------------------------
Step 5: Calculate End-use specific marginal damages
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Baseline Marginal Damages: WHOLE-HOME
-------------------------------------------------------------------------------------------------------
""")
# Make copies from scenario consumption to keep df smaller
print("\n", "Creating dataframe to store marginal damages calculations ...")
df_baseline_scenario_damages = df_euss_am_baseline_home.copy()

# calculate_marginal_damages(df, menu_mp, policy_scenario)
df_euss_am_baseline_home, df_baseline_scenario_damages = calculate_marginal_damages(df=df_euss_am_baseline_home,
                                                                                    menu_mp=menu_mp,
                                                                                    policy_scenario='No Inflation Reduction Act',
                                                                                    df_detailed_damages=df_baseline_scenario_damages
                                                                                    )
df_euss_am_baseline_home


-------------------------------------------------------------------------------------------------------
Step 5: Calculate End-use specific marginal damages
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Baseline Marginal Damages: WHOLE-HOME
-------------------------------------------------------------------------------------------------------


 Creating dataframe to store marginal damages calculations ...
-- Scenario: Baseline -- 
              scenario_prefix: 'baseline_', cambium_scenario: 'MidCase', emis_fossil_fuel_lookup: 'emis_fossil_fuel_lookup', 
              emis_climate_electricity_lookup: 'emis_preIRA_co2e_cambium21_lookup', damages_health_electricity_lookup: 'damages_preIRA_health_damages_lookup'
              
Available columns in df_copy: ['square_footage', 'census_region', 'census_division', 'census_div

,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,gea_region,state,city,county,...,baseline_2033_cooking_consumption,baseline_2034_cooking_consumption,baseline_2035_cooking_consumption,baseline_2036_cooking_consumption,baseline_2037_cooking_consumption,baseline_2038_cooking_consumption,baseline_heating_lifetime_damages_health,baseline_waterHeating_lifetime_damages_health,baseline_clothesDrying_lifetime_damages_health,baseline_cooking_lifetime_damages_health
bldg_id,,,,,,,,,,,,,,,,,,,,,
239,1690.0,South,East South Central,East South Central,Hot-Humid,90,SRSOc,AL,Not in a census Place,G0100390,...,979.44,979.44,979.44,979.44,979.44,979.44,540.78,210.55,55.52,3.92
273,1690.0,South,East South Central,East South Central,Mixed-Humid,90,SRSOc,AL,In another census Place,G0100150,...,537.20,537.20,537.20,537.20,537.20,537.20,800.98,285.62,130.91,65.35
307,1220.0,South,East South Central,East South Central,Hot-Humid,90,SRSOc,AL,Not in a census Place,G0100850,...,537.20,537.20,537.20,537.20,537.20,537.20,360.43,268.56,68.42,65.35
409,1220.0,South,East South Central,East South Central,Hot-Humid,90,SRSOc,AL,Not in a census Place,G0100050,...,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,0.00,0.00,50.80,4.06
517,1220.0,South,East South Central,East South Central,Mixed-Humid,89,SRSOc,AL,In another census Place,G0101270,...,377.18,377.18,377.18,377.18,377.18,377.18,547.70,119.46,44.41,45.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548109,1690.0,West,Mountain,Mountain North,Cold,23,RMPAc,WY,In another census Place,G5600050,...,405.61,405.61,405.61,405.61,405.61,405.61,4186.73,178.72,55.23,33.77
548226,2176.0,West,Mountain,Mountain North,Cold,23,RMPAc,WY,In another census Place,G5600050,...,405.61,405.61,405.61,405.61,405.61,405.61,0.00,0.00,29.14,33.77
548228,1690.0,West,Mountain,Mountain North,Cold,24,RMPAc,WY,Not in a census Place,G5600010,...,405.61,405.61,405.61,405.61,405.61,405.61,0.00,0.00,29.14,33.77


In [32]:
df_baseline_scenario_damages

,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,gea_region,state,city,county,...,baseline_2038_cooking_damages_climate_srmer,baseline_2038_cooking_damages_so2,baseline_2038_cooking_damages_nox,baseline_2038_cooking_damages_pm25,baseline_2038_cooking_damages_health,baseline_cooking_lifetime_tons_co2e_lrmer,baseline_cooking_lifetime_damages_climate_lrmer,baseline_cooking_lifetime_tons_co2e_srmer,baseline_cooking_lifetime_damages_climate_srmer,baseline_cooking_lifetime_damages_health
bldg_id,,,,,,,,,,,,,,,,,,,,,
239,1690.0,South,East South Central,East South Central,Hot-Humid,90,SRSOc,AL,Not in a census Place,G0100390,...,6.33,0.06,0.03,0.16,0.25,0.25,55.88,0.45,100.19,3.92
273,1690.0,South,East South Central,East South Central,Mixed-Humid,90,SRSOc,AL,In another census Place,G0100150,...,105.55,1.05,0.52,2.59,4.16,4.16,931.09,7.46,1669.56,65.35
307,1220.0,South,East South Central,East South Central,Hot-Humid,90,SRSOc,AL,Not in a census Place,G0100850,...,105.55,1.05,0.52,2.59,4.16,4.16,931.09,7.46,1669.56,65.35
409,1220.0,South,East South Central,East South Central,Hot-Humid,90,SRSOc,AL,Not in a census Place,G0100050,...,6.56,0.07,0.03,0.16,0.26,0.26,57.91,0.46,103.84,4.06
517,1220.0,South,East South Central,East South Central,Mixed-Humid,89,SRSOc,AL,In another census Place,G0101270,...,74.11,0.74,0.37,1.82,2.93,2.92,653.74,5.24,1172.24,45.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548109,1690.0,West,Mountain,Mountain North,Cold,23,RMPAc,WY,In another census Place,G5600050,...,71.80,0.86,0.27,0.97,2.10,1.85,414.04,5.19,1161.70,33.77
548226,2176.0,West,Mountain,Mountain North,Cold,23,RMPAc,WY,In another census Place,G5600050,...,71.80,0.86,0.27,0.97,2.10,1.85,414.04,5.19,1161.70,33.77
548228,1690.0,West,Mountain,Mountain North,Cold,24,RMPAc,WY,Not in a census Place,G5600010,...,71.80,0.86,0.27,0.97,2.10,1.85,414.04,5.19,1161.70,33.77


## Private Perspective: Annual Energy Costs

### Step 1: Obtain Level Energy Fuel Cost Data from the EIA
**Data Sources for Excel workbook containing state average Residential fuel cost for each fuel in 2018**
- EIA State Electricity Price: https://www.eia.gov/electricity/state/archive/2018/
- EIA Natural Gas Prices: https://www.eia.gov/dnav/ng/ng_pri_sum_dcu_SPA_a.htm
- Propane and Fuel Oil: EIA March 2023 Short Term Energy Outlook
    - https://www.eia.gov/outlooks/steo/pdf/wf01.pdf
    - Table WF01: Average Consumer Prices and Expenditures for Heating Fuels During the Winter
    - US Average: 2018-2019 Data

In [33]:
print("""
-------------------------------------------------------------------------------------------------------
Private Perspective: Annual Energy Costs
-------------------------------------------------------------------------------------------------------
- Step 1: Obtain Level Energy Fuel Cost Data from the EIA
- Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Step 1: Obtain Level Energy Fuel Cost Data from the EIA
-------------------------------------------------------------------------------------------------------
**Data Sources for Excel workbook containing state average Residential fuel cost for each fuel in 2018**
- EIA State Electricity Price: https://www.eia.gov/electricity/state/archive/2018/
- EIA Natural Gas Prices: https://www.eia.gov/dnav/ng/ng_pri_sum_dcu_SPA_a.htm
- Propane and Fuel Oil: EIA March 2023 Short Term Energy Outlook
    - https://www.eia.gov/outlooks/steo/pdf/wf01.pdf
    - Table WF01: Average Consumer Prices and Expenditures for Heating Fuels During the Winter
    - US Average: 2018-2019 Data
-------------------------------------------------------------------------------------------------------
""")

filename = 'fuel_prices_nominal.csv'
relative_path = os.path.join(r"fuel_prices", filename)
file_path = os.path.join(project_root, relative_path)
df_fuelPrices_perkWh = pd.read_csv(file_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# New units for the converted and inflated prices below
# $USD-2023, PREVIOUSLY USED $USD-2021
df_fuelPrices_perkWh['units'] = 'USD2022 per kWh'

years = ['2018', '2019', '2020', '2021', '2022']

# Take dataframe with nominal prices in their base units and convert to $/kWh equivalent
# https://www.eia.gov/energyexplained/units-and-calculators/british-thermal-units.php
for year in years:
    for index, row in df_fuelPrices_perkWh.iterrows():
        
        # Propane: (dollars per gallon) * (1 gallon propane/91,452 BTU) * (3412 BTU/1 kWh)
        if row['fuel_type'] == 'propane':
            df_fuelPrices_perkWh.at[index, f'{year}_fuelPrice_perkWh'] = row[f'{year}_nominal_unit_price'] * (1/91452) * (3412/1)
        
        # Fuel Oil: (dollars/gallon) * (1 gallon heating oil/138,500 BTU) * (3412 BTU/1 kWh)
        elif row['fuel_type'] == 'fuelOil':
            df_fuelPrices_perkWh.at[index, f'{year}_fuelPrice_perkWh'] = row[f'{year}_nominal_unit_price'] * (1/138500) * (3412/1)
        
        # Natural Gas: (dollars/cf) * (thousand cf/1000 cf) * (1 cf natural gas/1039 BTU) * (3412 BTU/1 kWh)
        elif row['fuel_type'] == 'naturalGas':
            df_fuelPrices_perkWh.at[index, f'{year}_fuelPrice_perkWh'] = row[f'{year}_nominal_unit_price'] * (1/1000) * (1/1039) * (3412/1)
        
        # Electricity: convert cents per kWh to $ per kWh
        elif row['fuel_type'] == 'electricity':
            df_fuelPrices_perkWh.at[index, f'{year}_fuelPrice_perkWh'] = row[f'{year}_nominal_unit_price'] / 100

# Convert nominal dollars to real 2022 US dollars (USD2022)
# $USD-2023, PREVIOUSLY USED $USD-2021
df_fuelPrices_perkWh['2018_fuelPrice_perkWh'] = df_fuelPrices_perkWh['2018_fuelPrice_perkWh'] * cpi_ratio_2023_2018
df_fuelPrices_perkWh['2019_fuelPrice_perkWh'] = df_fuelPrices_perkWh['2019_fuelPrice_perkWh'] * cpi_ratio_2023_2019
df_fuelPrices_perkWh['2020_fuelPrice_perkWh'] = df_fuelPrices_perkWh['2020_fuelPrice_perkWh'] * cpi_ratio_2023_2020
df_fuelPrices_perkWh['2021_fuelPrice_perkWh'] = df_fuelPrices_perkWh['2021_fuelPrice_perkWh'] * cpi_ratio_2023_2021
df_fuelPrices_perkWh['2022_fuelPrice_perkWh'] = df_fuelPrices_perkWh['2022_fuelPrice_perkWh'] * cpi_ratio_2023_2022

# Original dictionary mapping census divisions to states
map_states_census_divisions = {
    "New England": ["CT", "ME", "MA", "NH", "RI", "VT"],
    "Middle Atlantic": ["NJ", "NY", "PA"],
    "East North Central": ["IN", "IL", "MI", "OH", "WI"],
    "West North Central": ["IA", "KS", "MN", "MO", "NE", "ND", "SD"],
    "South Atlantic": ["DE", "DC", "FL", "GA", "MD", "NC", "SC", "VA", "WV"],
    "East South Central": ["AL", "KY", "MS", "TN"],
    "West South Central": ["AR", "LA", "OK", "TX"],
    "Mountain": ["AZ", "CO", "ID", "NM", "MT", "UT", "NV", "WY"],
    "Pacific": ["AK", "CA", "HI", "OR", "WA"]
}

# Reverse the mapping to create a state-to-census-division map
state_to_census_division = {}
for division, states in map_states_census_divisions.items():
    for state in states:
        state_to_census_division[state] = division

# Function to map location to census division
def map_location_to_census_division(location):
    if location in state_to_census_division:
        return state_to_census_division[location]
    return location

# Apply the function to map locations using .loc
df_fuelPrices_perkWh.loc[:, 'census_division'] = df_fuelPrices_perkWh['location_map'].apply(map_location_to_census_division)
# print(df_fuelPrices_perkWh)


-------------------------------------------------------------------------------------------------------
Private Perspective: Annual Energy Costs
-------------------------------------------------------------------------------------------------------
- Step 1: Obtain Level Energy Fuel Cost Data from the EIA
- Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Step 1: Obtain Level Energy Fuel Cost Data from the EIA
-------------------------------------------------------------------------------------------------------
**Data Sources for Excel workbook containing state average Residential fuel cost for each fuel in 2018**
- EIA State Electricity Price: https://www.eia.gov/electricity/state/archive/2018/
- EIA Natural Gas Prices: https://www.eia.gov/dnav/ng/ng_pri_sum_dcu_SPA_a.htm


In [34]:
# Project Fuel Prices from 2022 to 2050
filename = 'aeo_projections_2022_2050.xlsx'
relative_path = os.path.join(r"projections", filename)
file_path = os.path.join(project_root, relative_path)
df_fuelPrices_projection_factors = pd.read_excel(io=file_path, sheet_name='fuel_price_factors_2022_2050')

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
# print(df_fuelPrices_projection_factors)

# Convert the factors dataframe into a lookup dictionary including policy_scenario
factor_dict = df_fuelPrices_projection_factors.set_index(['region', 'fuel_type', 'policy_scenario']).to_dict('index')
# print(factor_dict)

Retrieved data for filename: aeo_projections_2022_2050.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\projections\aeo_projections_2022_2050.xlsx


In [35]:
# Pre-IRA policy_scenario: No Inflation Reduction Act
# Pass the desired policy_scenario as a parameter when applying the function
preIRA_projected_prices_df = df_fuelPrices_perkWh.apply(lambda row: project_future_prices(row, factor_dict, 'No Inflation Reduction Act'), axis=1)

# Concatenate the projected prices with the original DataFrame
df_fuelPrices_perkWh_preIRA = pd.concat([df_fuelPrices_perkWh, preIRA_projected_prices_df], axis=1)

# Create Fuel Price Lookup with the policy_scenario included
preIRA_fuel_price_lookup = create_fuel_price_lookup(df_fuelPrices_perkWh_preIRA, 'No Inflation Reduction Act')
# print(preIRA_fuel_price_lookup)


Processing location: National, fuel: naturalGas, policy_scenario: No Inflation Reduction Act
Initial price for 2022: 0.05043184441850111
Using projection factors for National, naturalGas, No Inflation Reduction Act: {2022: 1, 2023: 0.9755181241032465, 2024: 0.8896025800058641, 2025: 0.8412318775233192, 2026: 0.8086578203419481, 2027: 0.791217819099462, 2028: 0.7797455995454291, 2029: 0.7870977459578796, 2030: 0.7961259895028103, 2031: 0.804448058723705, 2032: 0.8090035617238965, 2033: 0.8169486175276889, 2034: 0.8214371689895306, 2035: 0.8249786745008596, 2036: 0.8264859586113565, 2037: 0.8345648475320383, 2038: 0.8507306007917632, 2039: 0.8655326974313067, 2040: 0.8655162568759125, 2041: 0.8670041621189876, 2042: 0.868808775422783, 2043: 0.869989067380462, 2044: 0.8679275616536569, 2045: 0.867906014032119, 2046: 0.8675913348058941, 2047: 0.8694052527644445, 2048: 0.8664931756653756, 2049: 0.8663586429503843, 2050: 0.867765954492123}
Year: 2022, Factor: 1, Future Price: 0.050431844418

In [36]:
# IRA-Reference policy_scenario: AEO2023 Reference Case
# Pass the desired policy_scenario as a parameter when applying the function
iraRef_projected_prices_df = df_fuelPrices_perkWh.apply(lambda row: project_future_prices(row, factor_dict, 'AEO2023 Reference Case'), axis=1)

# Concatenate the projected prices with the original DataFrame
df_fuelPrices_perkWh_iraRef = pd.concat([df_fuelPrices_perkWh, iraRef_projected_prices_df], axis=1)

# Create Fuel Price Lookup with the policy_scenario included
iraRef_fuel_price_lookup = create_fuel_price_lookup(df_fuelPrices_perkWh_iraRef, 'AEO2023 Reference Case')
# print(iraRef_fuel_price_lookup)


Processing location: National, fuel: naturalGas, policy_scenario: AEO2023 Reference Case
Initial price for 2022: 0.05043184441850111
Using projection factors for National, naturalGas, AEO2023 Reference Case: {2022: 1, 2023: 0.9839556270109511, 2024: 0.896931429801522, 2025: 0.8462875232065435, 2026: 0.8038662029823727, 2027: 0.7754636394030721, 2028: 0.7571439635072841, 2029: 0.7606949835527985, 2030: 0.7636889136297872, 2031: 0.7708472014082071, 2032: 0.7772290752131623, 2033: 0.7874921793677186, 2034: 0.7959209372991497, 2035: 0.8034350406721851, 2036: 0.8070259378095066, 2037: 0.8128780059719093, 2038: 0.8226579676773085, 2039: 0.8231216612992335, 2040: 0.8295963018124978, 2041: 0.8363521807627172, 2042: 0.8381048838873534, 2043: 0.8371994640239022, 2044: 0.8342263819703495, 2045: 0.8383924886670355, 2046: 0.860121585952518, 2047: 0.8590314022303608, 2048: 0.8567423872008247, 2049: 0.8570665110014251, 2050: 0.8594805442117667}
Year: 2022, Factor: 1, Future Price: 0.0504318444185011

### Step 2: Calculate Annual Operating (Fuel) Costs

### Baseline Fuel Cost: WHOLE-HOME

In [37]:
print("""
-------------------------------------------------------------------------------------------------------
Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
- Create a mapping dictionary for fuel types
- Create new merge columns to ensure a proper match.
- Merge df_copy with df_fuel_prices to get fuel prices for electricity, natural gas, propane, and fuel oil
- Calculate the per kWh fuel costs for each fuel type and region
- Calculate the baseline fuel cost 
-------------------------------------------------------------------------------------------------------
""")
# calculate_annual_fuelCost(df, menu_mp, policy_scenario, drop_fuel_cost_columns)
df_euss_am_baseline_home = calculate_annual_fuelCost(df=df_euss_am_baseline_home,
                                                     menu_mp=menu_mp,
                                                     policy_scenario='No Inflation Reduction Act',
                                                     drop_fuel_cost_columns=False
                                                     )
df_euss_am_baseline_home


-------------------------------------------------------------------------------------------------------
Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
- Create a mapping dictionary for fuel types
- Create new merge columns to ensure a proper match.
- Merge df_copy with df_fuel_prices to get fuel prices for electricity, natural gas, propane, and fuel oil
- Calculate the per kWh fuel costs for each fuel type and region
- Calculate the baseline fuel cost 
-------------------------------------------------------------------------------------------------------

Calculating BASELINE (no retrofit) fuel costs from 2024 to 2039 for heating
Calculating BASELINE (no retrofit) fuel costs from 2024 to 2036 for waterHeating
Calculating BASELINE (no retrofit) fuel costs from 2024 to 2037 for clothesDrying
Calculating BASELINE (no retrofit) fuel costs from 2024 to 2039 for cooking


,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,gea_region,state,city,county,...,baseline_2029_cooking_fuelCost,baseline_2030_cooking_fuelCost,baseline_2031_cooking_fuelCost,baseline_2032_cooking_fuelCost,baseline_2033_cooking_fuelCost,baseline_2034_cooking_fuelCost,baseline_2035_cooking_fuelCost,baseline_2036_cooking_fuelCost,baseline_2037_cooking_fuelCost,baseline_2038_cooking_fuelCost
bldg_id,,,,,,,,,,,,,,,,,,,,,
239,1690.0,South,East South Central,East South Central,Hot-Humid,90,SRSOc,AL,Not in a census Place,G0100390,...,53.04,53.84,54.43,54.95,55.63,56.10,56.49,56.75,57.40,57.94
273,1690.0,South,East South Central,East South Central,Mixed-Humid,90,SRSOc,AL,In another census Place,G0100150,...,69.94,70.12,70.68,71.02,71.04,71.14,71.04,70.89,71.21,71.34
307,1220.0,South,East South Central,East South Central,Hot-Humid,90,SRSOc,AL,Not in a census Place,G0100850,...,69.94,70.12,70.68,71.02,71.04,71.14,71.04,70.89,71.21,71.34
409,1220.0,South,East South Central,East South Central,Hot-Humid,90,SRSOc,AL,Not in a census Place,G0100050,...,54.67,55.50,56.11,56.64,57.34,57.82,58.23,58.50,59.17,59.73
517,1220.0,South,East South Central,East South Central,Mixed-Humid,89,SRSOc,AL,In another census Place,G0101270,...,49.11,49.23,49.63,49.86,49.88,49.95,49.88,49.77,50.00,50.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548109,1690.0,West,Mountain,Mountain North,Cold,23,RMPAc,WY,In another census Place,G5600050,...,44.12,44.61,45.36,45.95,46.28,46.52,46.62,47.06,47.46,47.71
548226,2176.0,West,Mountain,Mountain North,Cold,23,RMPAc,WY,In another census Place,G5600050,...,44.12,44.61,45.36,45.95,46.28,46.52,46.62,47.06,47.46,47.71
548228,1690.0,West,Mountain,Mountain North,Cold,24,RMPAc,WY,Not in a census Place,G5600010,...,44.12,44.61,45.36,45.95,46.28,46.52,46.62,47.06,47.46,47.71


# Model Runtime

In [42]:
# Get the current datetime again
end_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Calculate the elapsed time
elapsed_time = datetime.strptime(end_time, "%Y-%m-%d_%H-%M-%S") - datetime.strptime(start_time, "%Y-%m-%d_%H-%M-%S")

# Format the elapsed time
elapsed_seconds = elapsed_time.total_seconds()
elapsed_minutes = int(elapsed_seconds // 60)
elapsed_seconds = int(elapsed_seconds % 60)

# Print the elapsed time
print(f"The code took {elapsed_minutes} minutes and {elapsed_seconds} seconds to execute.")

The code took 4 minutes and 33 seconds to execute.
